# Try out Amazon Bedrock
In this short tutorial you prompt an LLM model on AWS Bedrock.

It includes different example use cases for Travel & Hospitality using Amazon Bedrock.

These sample use cases involve different tasks and prompt engeering techniques, as follows:

1. **Generate recommendations based on metadata**
- **Task**: Text Generation
- **Prompt Technique**: Zero-shot
2. **Estimate capacity for airlines or hotel properties based on historical data**
- **Task**: Complex Reasoning
- **Prompt Technique**: Chain-of-Thoughts (CoT)
3. **Create a question answering assistant for customer service**
- **Task**: Question Answering with Dialogue Asistant (without memory)
- **Prompt Technique**: Few-shot
4. **Summarize and classify content from media files transcription**
- **Task**: Text Summarization & Text Classification
- **Prompt Technique**: Zero-shot
5. **Create splash pages describing upcoming promotions**
- **Task**: Code Generation
- **Prompt Technique**: Zero-shot

Let's start by ensuring the Bedrock SDK is properly installed.

We'll also install a few libraries required in the notebook.

Let's start by ensuring the Bedrock SDK is properly installed.

We'll also install a few libraries required in the notebook.

In [ ]:
! pip install --upgrade boto3  # 1.36.21
! pip install --upgrade botocore


Now we can import the libraries and setup the Bedrock client.

In [ ]:
import json
from datetime import datetime

import boto3
import botocore

bedrock = boto3.client(service_name="bedrock", region_name="us-east-1")
bedrock_runtime = boto3.client('bedrock-runtime')

Let's get the list of Foundational Models supported in Bedrock at this time.

In [ ]:
bedrock.list_foundation_models()

We will define an utility function for calling Bedrock.

This will help passing the proper body depending on the model invoked, and will store the results in a CSV file as well.

In [ ]:
def call_bedrock(modelId, prompt_data):
    if 'amazon' in modelId:
        body = json.dumps({
            "inputText": prompt_data,
            "textGenerationConfig":
            {
                "maxTokenCount":4096,
                "stopSequences":[],
                "temperature":0,
                "topP":0.9
            }
        })
        #modelId = 'amazon.titan-text-lite-v1'
    elif 'anthropic' in modelId:
        body = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 4096,
            "temperature": 0,
            "top_k":250,
            "top_p":0.9,
            "messages": [
                {
                    "role": "user",
                    "content": [{"type": "text", "text": prompt_data}]
                }
            ],
        })
        #modelId = 'anthropic.claude-v2:1'
    else:
        print('Parameter model must be one of amazon titan or anthropic claude')
        return
    accept = 'application/json'
    contentType = 'application/json'

    before = datetime.now()
    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    latency = (datetime.now() - before)
    response_body = json.loads(response.get('body').read())
    
    if 'amazon' in modelId:
        response = response_body.get('results')[0].get('outputText')
    elif 'anthropic' in modelId:
        response = response_body.get('content')[0].get('text')
    return response, latency

Now we are ready for running our examples with different models.

-----

## 1. Generate recommendations based on metadata

**Use Case:** A company wants to generate recommendations of flight destinations for their users based on some metadata, e.g. country, age-range, and interests.

**Task:** Text Generation

**Prompt Technique:** Zero-shot

In [ ]:
prompt_data ="""
Human:
Generate a list of 10 recommended Melia Hotels for traveling considering the information in the <metadata></metadata> XML tags, and include a very brief description of each recommendation.
Answer in English.

<metadata>
Traveler going to Spain
Age range between 20-30
Interested on water sports and theme parks
Traveling during the summer
</metadata>

Assistant:
"""

In [ ]:
response, latency = call_bedrock('anthropic.claude-v2:1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

------

## 2. Estimate capacity for airlines or hotel properties based on historical data

**Use Case:** A T&H company wants to estimate capacity/occupancy levels they could have for the next days based on historical information and flights/hotels metadata.

**Task:** Complex Reasoning

**Prompt Technique:** Chain-of-Thoughts (CoT)

In [ ]:
prompt_data ="""
Human: Last week, the number of guests at 3 Melia hotels was as follows:
- Monday: Paris 650, New York 320, Singapore 415
- Tuesday: Paris 640, New York 330, Singapore 410
- Wednesday: Paris 630, New York 340, Singapore 425

Question: How many guests can we expect next Friday at the Paris hotel?
Answer: Based on the numbers given and without any further information, there is a daily decrease of 10 guests for the Paris hotel.
If we assume that this trend will continue for the next few days, we can expect 620 guests for the next day, which is Thursday, and
therefore, 610 guests for Friday.

Question: How many guests can we expect on Saturday at each of the hotels?
Give a step-by-step reasoning and recommendations for increasing the number of guests at the hotels.
Assistant:
"""

In [ ]:
response, latency = call_bedrock('anthropic.claude-v2:1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

------

## 3. Create a question answering assistant for customer service

**Use Case:** A company wants to create a bot capable of answering questions about the services available, based on the internal information for these.

**Task:** Question Answering with Dialogue Asistant (no memory)

**Prompt Technique:** Few-shot

In [ ]:
prompt_data ="""
Context: An airline services available for purchase are as follows
1. Seat upgrades, available from 20 Euros, to classes Economy Plus and Business, on weekdays' flights
2. Meals, available for payment in-flight, mediterranean menu, on all days' flights
3. Fast boarding, from 30 Euros, available for premier customers, on all days' flights

Instruction: Answer any questions about the services available in a friendly manner. If you don't know the answer just say 'Apologies, but I don't have the answer for that. Please contact our team by phone.'

Assistant: Welcome to Airline Services, how can I help you?
Human: Hi, I would like to know what are the services available please.
Assistant: Of course, right now we have the seat upgrades, the meals, and the fast boarding.
Human: Thank you. I would like to know details for those please.
Assistant:
"""

In [ ]:
response, latency = call_bedrock('anthropic.claude-v2:1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

------

## 4. Generate content summary based on transcriptions from media files

**Use Case:** A company needs to generate summaries of the audio transcription for audio and video files for customer service, to be sent to their operations quality team. They also want to classify this content according to specific categories.

**Task:** Text Summarization & Text Classification

**Prompt Technique:** Zero-shot

#### (Pre-requisite)

First, we will need to transcribe the media files. You can e.g. use Amazon Transcribe for this task following examples like this: https://docs.aws.amazon.com/code-library/latest/ug/transcribe_example_transcribe_StartTranscriptionJob_section.html

For our sample we will start from an interview transcription in the file "interview.txt".

In [ ]:
f = open("interview.txt", "r").read()
print(f)

In [ ]:
prompt_data =f"""
Human:
Generate a summary of the transcription in the <transcription></transcription> XML tags below.
Then, classify the mood of the participants according to the closest category within 'fully satisfied', 'satisfied', 'unsatisfied', 'fully unsatisfied', or 'neutral'.

<transcription>
{f}
</transcription>

Assistant:
"""

In [ ]:
response, latency = call_bedrock('anthropic.claude-v2', prompt_data)
print(response, "\n\n", "Inference time:", latency)

In [ ]:
response, latency = call_bedrock('anthropic.claude-instant-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

------

## 5. Create splash pages describing upcoming promotions

**Use Case:** A company wants to create HTML pages quickly and easily for their upcoming promotions.

**Task:** Code Generation

**Prompt Technique:** Zero-shot

In [ ]:
prompt_data ="""
There is an upcoming promotion presented by the Melia Salou Hotel.
The promotion is targeting young audience in the age range between 18 and 40.
The promotion consists of a 20% discount when booking rooms online.
There will be additional fees for upgrades and bookings can be done trought this same portal.
The promotion is part of the Summer Discounts of the company.
The promotion is available from June 28th to August 31st.

Based the this information, generate the HTML code for an attractive splash page for this promotion.
Include catchy phrases and invite customers to sign-up for the Melia Hotel's loyalty program.
Have the splash page use blue fonts and black background, according to the hotel's branding.
"""

In [ ]:
response, latency = call_bedrock('anthropic.claude-v2', prompt_data)
#print(response, "\n\n", "Inference time:", latency)
from IPython.display import display, HTML
display(HTML(response))

In [ ]:
response, latency = call_bedrock('anthropic.claude-instant-v1', prompt_data)
#print(response, "\n\n", "Inference time:", latency)
from IPython.display import display, HTML
display(HTML(response))

------